In [3]:
import os
import sys
import glob
from operator import index
from os import write

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from traitsui.examples.demo.Advanced.Tabular_editor_with_context_menu_demo import columns

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities
from iDrink.iDrinkUtilities import get_title_measure_name, get_unit, get_cad, get_setting_axis_name

In [4]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')

df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']
idx_s_singlecam_full = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']
idx_s_singlecam = ['S017', 'S018']
idx_s_multicam = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']
idx_s_multicam_reduced = ['S001', 'S002']
idx_s_reduced = idx_s_multicam_reduced + idx_s_singlecam
idx_s = idx_s_multicam + idx_s_singlecam

# Range of Motion Error
## Time-Series

In [5]:
from scipy import stats

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots = os.path.join(dir_stat_cont, '02_plots')

csv_rom = os.path.join(dir_results, 'omc_mmc_rom.csv')
df_rom = pd.read_csv(csv_rom, sep=';')
df_rom.dropna(axis=1, inplace=True)
dir_boxplots_rom_error = os.path.join(dir_plots, '04_rom', '01_rom_error_boxplots')

df_rom = df_rom[(df_rom['id_s'].isin(idx_s_reduced)) & (~df_rom['id_p'].isin(ignore_id_p))]

# Detect outlier for ROM errors
cols_error = ['hand_vel_rom_error', 'elbow_vel_rom_error', 'trunk_disp_rom_error', 'elbow_flex_pos_rom_error', 'shoulder_flex_pos_rom_error', 'shoulder_abduction_pos_rom_error']
df_rom = df_rom[(np.abs(stats.zscore(df_rom[cols_error])) < 3).all(axis=1)]


#metrics = ['hand_vel', 'elbow_vel', 'trunk_disp', 'trunk_ang', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']
metrics = ['hand_vel', 'elbow_vel', 'trunk_disp', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

df_filtered = df_rom[~df_rom['id_p'].isin(ignore_id_p)].sort_values(by='id_p')
df_filtered = df_filtered[df_filtered['dynamic'] == 'fixed']
df_filtered_rom_errors = df_filtered.groupby(['id_s', 'id_t']).median(numeric_only=True).filter(like='error')

df_for_plot = pd.DataFrame(columns=['id_s', 'id_s_name', 'id_t', 'measure', 'error'])
joint_metrics = ['elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

for id in df_filtered_rom_errors.index.to_list():
    id_s = id[0]
    id_t = id[1]
    id_s_name = get_setting_axis_name(id_s)
    for metric in metrics:
        error = df_filtered_rom_errors.loc[id, f'{metric}_rom_error']
        df_for_plot = pd.concat([df_for_plot, pd.DataFrame({'id_s': id_s, 'id_s_name': id_s_name, 'id_t': id_t, 'measure': metric, 'error': error}, index=[0])], ignore_index=True)
    

In [6]:
os.makedirs(dir_boxplots_rom_error, exist_ok=True)

for measure in metrics:
    unit = get_unit(measure)
    title_name = get_title_measure_name(measure)
        
    cad = get_cad(df_cad, measure)
    cad_neg = -cad
    
    fig = px.box(df_for_plot[df_for_plot['measure'] == measure].sort_values(by='id_s'), x='id_s_name', y='error', title=f'{title_name} ROM error, cad = {cad}', template='plotly', hover_name='id_t', width=650, height=650)
    

    
    fig.add_hline(y=cad, line_dash='dash', line_color='red', name='CAD')
    fig.add_hline(y=cad_neg, line_dash='dash', line_color='red', name='CAD')
    
    fig.update_layout(title= dict(text=f'<b>{title_name} ROM error<br>cad = {cad} {unit}<b>', font=dict(size=26)),
                      xaxis_title=dict(text=f'<b>Settings<b>', font=dict(size=22)),
                      yaxis_title=dict(text=f'<b>ROM error [{unit}]<b>', font=dict(size=22)),
                      xaxis=dict(tickfont=dict(size=20))
                  )
    
    #fig.show()

    os.makedirs(os.path.join(dir_boxplots_rom_error, 'png'), exist_ok=True)
    
    filepath = os.path.join(dir_boxplots_rom_error, f'{measure}_rom_error_boxplot.html')
    fig.write_html(filepath)
    
    filepath = os.path.join(dir_boxplots_rom_error, 'png', f'0304_{measure}_rom_error_boxplot.png')
    fig.write_image(filepath, scale=5)
    
    filepath = os.path.join(dir_boxplots_rom_error, f'{measure}_rom_error_boxplot.svg')
    fig.write_image(filepath)
    

In [7]:
ignore_id_s = ['S003', 'S005', 'S006', 'S007', 'S008']

In [8]:
df_filtered = df_rom[~df_rom['id_p'].isin(ignore_id_p)].sort_values(by='id_p')
df_filtered = df_filtered[~df_filtered['id_s'].isin(ignore_id_s)].sort_values(by='id_p')


df_filtered = df_filtered[df_filtered['dynamic'] == 'fixed']
df_filtered_rom_errors = df_filtered.groupby(['id_s', 'id_t']).mean(numeric_only=True).filter(like='error')

df_for_plot_corr = pd.DataFrame(columns=['id_s', 'id_s_name', 'id_t', 'measure', 'error'])
joint_metrics = ['elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

for id in df_filtered_rom_errors.index.to_list():
    id_s = id[0]
    id_t = id[1]
    id_s_name = get_setting_axis_name(id_s)
    for metric in metrics:
        error = df_filtered_rom_errors.loc[id, f'{metric}_rom_error']
        df_for_plot_corr = pd.concat([df_for_plot_corr, pd.DataFrame({'id_s': id_s, 'id_s_name': id_s_name, 'id_t': id_t, 'measure': metric, 'error': error}, index=[0])], ignore_index=True)
    

In [9]:
df_checkup = df_for_plot.groupby(['id_s', 'measure']).median(numeric_only=True)

In [10]:
os.makedirs(dir_boxplots_rom_error, exist_ok=True)

for measure in metrics:
    unit = get_unit(measure)
    title_name = get_title_measure_name(measure)
    
    cad = get_cad(df_cad, measure)
    cad_minus = -cad
    
    fig = px.box(df_for_plot_corr[df_for_plot_corr['measure'] == measure], x='id_s_name', y='error', title=f'{title_name} ROM error, cad = {cad} {unit}',
                 template='plotly')
    

    
    fig.add_hline(y=cad, line_dash='dash', line_color='red', name='CAD')
    fig.add_hline(y=-cad, line_dash='dash', line_color='red', name='CAD')
    
    fig.update_layout(title= f'{title_name} ROM error, cad = {cad} {unit}',
                  xaxis_title=f'Settings',
                  yaxis_title=f'ROM error [{unit}]',
                  )
    
    #fig.show()
    
    """filepath = os.path.join(dir_boxplots_rom_error, f'{measure}_rom_error_boxplot_corr.html')
    fig.write_html(filepath)
    
    filepath = os.path.join(dir_boxplots_rom_error, f'{measure}_rom_error_boxplot_corr.png')
    fig.write_image(filepath)"""

## ROM RMSE


In [11]:
dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots = os.path.join(dir_stat_cont, '02_plots')

csv_rom_rmse = os.path.join(dir_results, 'omc_mmc_rom_rmse.csv')
df_rom_rmse = pd.read_csv(csv_rom_rmse, sep=';')

metrics = ['hand_vel', 'elbow_vel', 'trunk_disp', 'elbow_flex_pos', 'shoulder_flex_pos',
           'shoulder_abduction_pos']

In [12]:
"""df_rom_rmse['id_s']= df_rom_rmse['id'].str.split('_').str[0]
df_rom_rmse['id_p']= df_rom_rmse['id'].str.split('_').str[1]
df_rom_rmse['id_p'] = df_rom_rmse['id_p'].fillna(np.nan)

df_rom_rmse = df_rom_rmse.drop(columns=['id'])"""

"df_rom_rmse['id_s']= df_rom_rmse['id'].str.split('_').str[0]\ndf_rom_rmse['id_p']= df_rom_rmse['id'].str.split('_').str[1]\ndf_rom_rmse['id_p'] = df_rom_rmse['id_p'].fillna(np.nan)\n\ndf_rom_rmse = df_rom_rmse.drop(columns=['id'])"

In [13]:
df_rom_rmse

,id,dynamic,condition,hand_vel_rom_rmse,elbow_vel_rom_rmse,trunk_disp_rom_rmse,trunk_ang_rom_rmse,elbow_flex_pos_rom_rmse,shoulder_flex_pos_rom_rmse,shoulder_abduction_pos_rom_rmse,...,elbow_flex_pos_max_rmse,elbow_flex_pos_min_rmse,shoulder_flex_pos_max_rmse,shoulder_flex_pos_min_rmse,shoulder_abduction_pos_max_rmse,shoulder_abduction_pos_min_rmse,id_s,id_p,metric,rom_rmse
0,S003,fixed,unaffected,134.501333,1.532447,63.128792,2.192513,4.226828,6.956935,10.481980,...,3.759723,2.112327,7.664804,1.553764,3.745772,8.643486,NaN,NaN,NaN,NaN
1,S003,fixed,affected,267.889004,1.332489,3.892989,3.388246,11.156660,11.327910,3.450634,...,2.576818,9.384753,14.924954,3.963280,8.359042,9.764519,NaN,NaN,NaN,NaN
2,S003,dynamic,unaffected,134.501333,1.532447,63.128792,2.192513,4.226828,6.956935,10.481980,...,7.009061,3.586135,2.141911,5.072722,5.428947,5.320057,NaN,NaN,NaN,NaN
3,S003,dynamic,affected,267.891432,1.332479,3.890900,3.388246,11.156660,11.327837,3.452875,...,8.689380,3.048534,8.450370,3.034263,3.822708,4.867207,NaN,NaN,NaN,NaN
4,S004,fixed,unaffected,1137.507509,1.525185,63.400222,2.201264,4.219640,6.929598,10.362900,...,3.769273,2.109316,7.660595,1.576040,3.687245,8.602983,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,NaN,dynamic,unaffected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,S010,P07,shoulder_flex_pos,19.211172
284,NaN,fixed,affected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,S010,P07,shoulder_abduction_pos,5.998121
285,NaN,fixed,unaffected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,S010,P07,shoulder_abduction_pos,7.760618
286,NaN,dynamic,affected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,S010,P07,shoulder_abduction_pos,5.998121


In [14]:
# Detect outlier for ROM errors
cols_error = ['hand_vel_rom_error', 'elbow_vel_rom_error', 'trunk_disp_rom_error', 'trunk_ang_rom_error', 'elbow_flex_pos_rom_error', 'shoulder_flex_pos_rom_error', 'shoulder_abduction_pos_rom_error']
df_rom = df_rom[(np.abs(stats.zscore(df_rom[cols_error])) < 3).all(axis=1)]

df_rom_rmse.dropna(axis=0, inplace=True)
dir_barplots_rom_rmse_error = os.path.join(dir_plots, '04_rom', '02_rom_rmse_barplots')

"""for metric in metrics:
    mask = np.abs(stats.zscore(df_rom_rmse[df_rom_rmse['metric'] == metric]['rom_rmse'])) < 3
    mask = mask.reindex(df_rom_rmse.index, fill_value=True)
    df_rom_rmse = df_rom_rmse.loc[mask]"""

df_filtered = df_rom_rmse[(df_rom_rmse['id_s'].isin(idx_s_reduced))&(~df_rom_rmse['id_p'].isin(ignore_id_p))].sort_values(by='id_p')
df_filtered = df_filtered[df_filtered['dynamic'] == 'fixed']
df_filtered = df_filtered.groupby(['id_s', 'metric']).mean(numeric_only=True)

df_for_plot = pd.DataFrame(columns=['id_s', 'id_s_name',  'metric', 'rom_rmse'])
joint_metrics = ['elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

for id in df_filtered.index.to_list():
    id_s = id[0]
    metric = id[1]
    rmse = df_filtered.loc[id, 'rom_rmse']
    id_s_name = get_setting_axis_name(id_s)
    df_for_plot = pd.concat([df_for_plot, pd.DataFrame({'id_s': id_s, 'id_s_name': id_s_name, 'metric': metric, 'rom_rmse': rmse}, index=[0])], ignore_index=True)

In [15]:
os.makedirs(dir_barplots_rom_rmse_error, exist_ok=True)

for metric in metrics:
    cad = get_cad(df_cad, metric)
    unit = get_unit(metric)
    title_name = get_title_measure_name(metric)
    fig = px.bar(df_for_plot[df_for_plot['metric'] == metric], x='id_s_name', y='rom_rmse', title=f'{title_name} ROM RMSE, cad = {cad}',
                 template='plotly', width=650, height=650)
    
    
    
    fig.add_hline(y=cad, line_dash='dash', line_color='red', name='CAD')

    fig.update_layout(title= dict(text=f'<b>{title_name} ROM RMSE,<br>cad = {cad} {unit}<b>', font=dict(size=26)),
              xaxis_title=dict(text=f'<b>Settings<b>', font=dict(size=22)),
              yaxis_title=dict(text=f'<b>ROM RMSE [{unit}]<b>', font=dict(size=22)),
                  xaxis=dict(tickfont=dict(size=20))
              )
    #fig.show()
    filepath = os.path.join(dir_barplots_rom_rmse_error, f'{metric}_rom_rmse_barplot.html')
    fig.write_html(filepath)

    os.makedirs(os.path.join(dir_barplots_rom_rmse_error, 'png'), exist_ok=True)
    filepath = os.path.join(dir_barplots_rom_rmse_error, 'png', f'0305-{metric}_rom_rmse_barplot.png')
    fig.write_image(filepath)

In [122]:
df_for_plot.insert(0, 'Setting', df_for_plot['id_s'].apply(lambda x: get_setting_axis_name(x)))
df_for_plot.drop(columns=['id_s_name', 'id_s'], inplace=True)



In [123]:
df_for_plot['CAD'] = df_for_plot['metric'].apply(lambda x: get_cad(df_cad, x))
df_for_plot['metric'] = df_for_plot['metric'].apply(lambda x: get_title_measure_name(x, add_unit = True))

df_for_plot['RMSE < CAD'] = df_for_plot.apply(
    lambda row: 'Yes' if (row['rom_rmse'] < row['CAD']) else 'No', axis=1)

In [124]:
df_latex = df_for_plot.round(2)

In [86]:
df_latex = df_for_plot.groupby(['Setting', 'metric']).mean(numeric_only=True).round(2)

In [125]:
print(df_latex.style.to_latex(label='tab:res:cont:rom_rmse',
                              caption=r'ROM RMSE for 5-cam and single-cam settings. The full table can be found in section \ref{sec:app:plots_ts}'))

\begin{table}
\caption{ROM RMSE for 5-cam and single-cam settings. The full table can be found in section \ref{sec:app:plots_ts}}
\label{tab:res:cont:rom_rmse}
\begin{tabular}{lllrrl}
 & Setting & metric & rom_rmse & CAD & RMSE < CAD \\
0 & SimCC, Cams: 1,2,3,4,5 & Elbow Flexion [deg] & 13.370000 & 4.480000 & No \\
1 & SimCC, Cams: 1,2,3,4,5 & Elbow Velocity [deg/s] & 3.170000 & 21.710000 & Yes \\
2 & SimCC, Cams: 1,2,3,4,5 & Hand Velocity [mm/s] & 5465.320000 & 78.960000 & No \\
3 & SimCC, Cams: 1,2,3,4,5 & Shoulder Abduction [deg] & 6.820000 & 9.990000 & Yes \\
4 & SimCC, Cams: 1,2,3,4,5 & Shoulder Flexion [deg] & 12.520000 & 7.820000 & No \\
5 & SimCC, Cams: 1,2,3,4,5 & Trunk Angle [deg] & 25.170000 & nan & No \\
6 & SimCC, Cams: 1,2,3,4,5 & Trunk Displacement [mm] & 40.410000 & 31.890000 & No \\
7 & Metrabs, Cams: 1,2,3,4,5 & Elbow Flexion [deg] & 9.090000 & 4.480000 & No \\
8 & Metrabs, Cams: 1,2,3,4,5 & Elbow Velocity [deg/s] & 2.600000 & 21.710000 & Yes \\
9 & Metrabs, Cams: 1,2

In [81]:
df_for_plot.groupby(['id_s', 'metric']).median(numeric_only=True)

KeyError: 'id_s'

In [21]:
df_filtered = df_rom_rmse[df_rom_rmse['id_s'].isin(idx_s)].sort_values(by='id_p')
df_rom_rmse_corr = df_filtered.groupby(['id_s', 'metric']).mean(numeric_only=True)

df_for_plot_corr_rmse = pd.DataFrame(columns=['id_s', 'metric', 'rom_rmse'])
joint_metrics = ['elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

for id in df_rom_rmse_corr.index.to_list():
    id_s = id[0]
    metric = id[1]
    rmse = df_rom_rmse_corr.loc[id, 'rom_rmse']
    df_for_plot_corr_rmse = pd.concat([df_for_plot_corr_rmse, pd.DataFrame({'id_s': [id_s], 'metric': [metric], 'rom_rmse': [rmse]})], ignore_index=True)

df_for_plot_corr_rmse['ids_name'] = df_for_plot_corr_rmse['id_s'].apply(lambda x: get_setting_axis_name(x))

os.makedirs(dir_barplots_rom_rmse_error, exist_ok=True)

for metric in metrics:
    cad = get_cad(df_cad, metric)
    unit = get_unit(metric)
    title_name = get_title_measure_name(metric)
    fig = px.bar(df_for_plot_corr_rmse[df_for_plot_corr_rmse['metric'] == metric].sort_values(by='id_s'), x='ids_name', y='rom_rmse', title=f'{title_name} ROM RMSE, cad = {cad}',
                 template='plotly')
    
    fig.add_hline(y=cad, line_dash='dash', line_color='red', name='CAD')
    
    fig.update_yaxes(range=[0,  min(2500, df_for_plot_corr_rmse[df_for_plot_corr_rmse['metric'] == metric]['rom_rmse'].max()) + 10])
    
    fig.update_layout(title= dict(text=f'{title_name} ROM RMSE, cad = {cad} {unit}', font=dict(size=24)),
                  xaxis_title=dict(text=f'Settings', font=dict(size=20)),
                  yaxis_title=dict(text=f'ROM RMSE [{unit}]', font=dict(size=20)),
                  )
    fig.show()
    
    newdir = os.path.join(dir_barplots_rom_rmse_error , 'new')
    os.makedirs(newdir, exist_ok=True)
    filepath = os.path.join(dir_barplots_rom_rmse_error, f'{metric}_rom_rmse_barplot_corr.html')
    fig.write_html(filepath)
    
    filepath = os.path.join(newdir, f'0601_{metric}_rom_rmse_barplot_corr.png')
    fig.write_image(filepath)

In [16]:
df_for_plot_corr_rmse

,id_s,metric,rom_rmse
0,S001,elbow_flex_pos,16.627595
1,S001,elbow_vel,3.992229
2,S001,hand_vel,10891.368523
3,S001,shoulder_abduction_pos,13.058625
4,S001,shoulder_flex_pos,15.612322
...,...,...,...
142,S026,hand_vel,1080.370298
143,S026,shoulder_abduction_pos,2.940550
144,S026,shoulder_flex_pos,4.294567
145,S026,trunk_ang,3.881742


In [17]:
df_for_plot_corr_rmse.groupby(['id_s', 'metric']).mean(numeric_only=True)

rom_rmse
id_s metric                              
S001 elbow_flex_pos             16.627595
     elbow_vel                   3.992229
     hand_vel                10891.368523
     shoulder_abduction_pos     13.058625
     shoulder_flex_pos          15.612322
...                                   ...
S026 hand_vel                 1080.370298
     shoulder_abduction_pos      2.940550
     shoulder_flex_pos           4.294567
     trunk_ang                   3.881742
     trunk_disp                 37.548138

[147 rows x 1 columns]

# Correlation
